This notebook deals with extracting land surface temperature in an area (in our case the impacted area) 

In [ ]:
from google.colab import drive#Authenticating to google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import colorcet as cc
import folium

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=OQDI68xzApT4LiMykKcsh3__tE9H2Wdvu2FnpRCGRhI&tc=65C8X5VpgUlbMkvm8rmHZpcMTLPfuirwKKu-mKm7QHw&cc=JzqPaOg3CEhbKolZ2ID-DKkbUSPCnJUXlBjlaZIm6fM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjb-8qHgvKP-UsQpp98X_A-iLEiDEMUG0PFdzoADtc-UKOxCCnmLpg

Successfully saved authorization token.


In [ ]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

In [ ]:
# Initial date of interest (inclusive).
i_date = '2021-08-27'

# Final date of interest (exclusive).
f_date = '2021-08-28'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [ ]:
# Define the urban location of interest as a point near Lyon, France.
lat, lon = 38.829592, 23.344883

point = ee.Geometry.Point(lon, lat)


In [ ]:
scale = 1000  # scale in meters

# Print the elevation 
elv_point = elv.sample(point, scale).first().get('elevation').getInfo()
print('Ground elevation', elv_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_point = lst.mean().sample(point, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST:', round(lst_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_point = lc.first().sample(point, scale).first().get('LC_Type1').getInfo()
print('Land cover value :', lc_point)

In [ ]:
# Get the data for the pixel intersecting the point in urban area.
lst_point= lst.getRegion(point, scale).getInfo()



# Preview the result.
lst_point[:5]

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km', 'QC_Day'],
 ['2021_08_27',
  23.342722657845766,
  38.829678156066315,
  1630022400000,
  15749,
  0]]

In [ ]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [ ]:
lst_df = ee_array_to_df(lst_u_poi,['LST_Day_1km'])

def t_modis_to_celsius(t_modis):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius =  0.02*t_modis - 273.15
    return t_celsius
temp=t_modis_to_celsius
# Apply the function to get temperature in celsius.
lst_df['LST_Day_1km'] = lst_df['LST_Day_1km'].apply(temp)

lst_df.head()

TEMPERATURE: <function t_modis_to_celsius at 0x7f1a9c4d0320>


,time,datetime,LST_Day_1km
0,1630022400000,2021-08-27,41.83


In [ ]:
# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

In [ ]:
# Define a region of interest with a buffer zone of 1000 km .
roi = u_poi.buffer(1e6)

In [ ]:
from IPython.display import Image

# Create a URL to the styled image for a region 
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']})
print(url)

# Display the thumbnail land surface temperature
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/10b7183d16311948787beba78aa769d9-85380331f2570637f2a55cc0c1eff490:getPixels

Please wait while the thumbnail loads, it may take a moment...
